## このノートブックでは、提出用のシンプルなベースラインを作成し、今後詳しく分析するために重要な特徴量を特定します。

In [ ]:
# pythonのバージョン確認
!python --version

In [ ]:
-- py_packages stageの確認
LIST @py_packages;

In [1]:
# 必要なライブラリの読み込み
import numpy as np
import pandas as pd

from catboost import CatBoostRegressor, Pool
import matplotlib.pyplot as plt
import seaborn as sns

import gc

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
raw_train_df = session.table("train").to_pandas()
raw_test_df = session.table("test").to_pandas()

In [4]:
raw_train_df.shape, raw_test_df.shape

((363924, 149), (112437, 149))

In [5]:
set(raw_train_df.columns) - set(raw_test_df.columns)

{'money_room'}

In [6]:
set(raw_test_df.columns)- set(raw_train_df.columns) 

{'id'}

 ###  money_roomが目的変数で、'id'列は識別子です。その他の列は両方のデータセットで同じです。

In [ ]:
# 列名の確認
raw_train_df.columns

In [7]:
train_money_room = raw_train_df["MONEY_ROOM"].values
test_id = raw_test_df['ID']

In [ ]:
train_df = session.table("processed_train_1218").to_pandas()
test_df = session.table("processed_test_1218").to_pandas()

In [ ]:
tmp = [
    "unit_count",
    "lon",
    "lat",
    "total_floor_area",
    "building_area",
    "floor_count",
    "basement_floor_count",
    "building_land_area",
    "land_area_all",
    "unit_area_min",
    "unit_area_max",
    "land_setback",
    "land_kenpei",
    "land_youseki",
    "room_floor",
    "balcony_area",
    "room_count",
    "unit_area",
    "empty_number",
    "nl",
    "el",
    "bus_time1",
    "walk_distance1",
    "bus_time2",
    "walk_distance2",
    "traffic_car",
    "snapshot_land_area",
    "snapshot_land_shidou",
    "land_shidou_a",
    "land_shidou_b",
    "land_mochibun_a",
    "land_mochibun_b",
    "house_area",
    "room_kaisuu",
    "madori_number_all",
    "money_kyoueki",
    "money_rimawari_now",
    "money_shuuzen",
    "money_shuuzenkikin",
    "money_sonota1",
    "money_sonota2",
    "money_sonota3",
    "parking_money",
    "parking_distance",
    "parking_number",
    "school_ele_distance",
    "school_jun_distance",
    "convenience_distance",
    "super_distance",
    "hospital_distance",
    "park_distance",
    "drugstore_distance",
    "bank_distance",
    "shopping_street_distance",
    "est_other_distance",
]
numerical_columns = [col.upper() for col in tmp]

In [36]:
categorical_cols = [c for c in train_df.columns if c not in numerical_columns]

In [ ]:
train_df[categorical_cols] = train_df[categorical_cols].fillna("NAN")

In [41]:
train_y = np.log1p(train_money_room)

In [ ]:
params = {
    "loss_function": "MAE",
    "task_type":  "GPU",  # CPUで実行する場合はこれを削除
    "random_seed": 2025,
    "verbose": 1000,
    "iterations": 1
}

In [44]:
from catboost import CatBoostRegressor


model = CatBoostRegressor(**params) 
model.fit(train_df, train_y, cat_features=categorical_cols)

0:	learn: 0.4886150	total: 1.68s	remaining: 0us


In [ ]:
import pickle

model_path = "/tmp/model.pkl"

with open(model_path, "wb") as f:
    pickle.dump(model, f)

In [ ]:
stage_name = "@model"

session.sql(f"""
    PUT file://{model_path} {stage_name}
    OVERWRITE = TRUE
""").collect()

In [ ]:
# # 読み込み用
# with open("../models/catboost_model.pkl", "rb") as f:
#     loaded_model = pickle.load(f)

# # 推論例
# pred = loaded_model.predict(test_df)
# print(pred[:10])

[16.88302386 16.90028201 16.88814303 16.89601399 16.88302386 16.90129069
 16.88302386 16.89601399 16.90129069 16.89601399]


In [57]:
train_pred = np.expm1(model.predict(train_df))

In [46]:
from sklearn.metrics import mean_absolute_percentage_error


mape_error = mean_absolute_percentage_error(train_money_room, train_pred)
print("MAPE Error ", mape_error)

MAPE Error  0.5678790285270185


In [47]:
feature_importance = pd.DataFrame({'feature_name': train_df.columns,
                            'importance': model.get_feature_importance()})

# feature_importance = feature_importance[feature_importance["importance"]>0]

In [49]:
#  今後の分析のために保存します。
feature_importance_path = "/tmp/feature_importance.csv"
feature_importance.to_csv(feature_importance_path, index=False)

In [50]:
feature_importance.sort_values("importance", ascending=False)[:20]

,feature_name,importance
18,year_built,30.064671
100,madori_kind_all,26.369284
136,City/town/village name,20.825550
94,house_area,14.297417
3,building_create_date,8.443078


In [ ]:
stage_name = "@model"

session.sql(f"""
    PUT file://{feature_importance_path} {stage_name}
    OVERWRITE = TRUE
""").collect()

# 提出用ファイルを作成する

In [51]:
test_pred = model.predict(test_df)
test_pred = np.expm1(test_pred)

In [52]:
submit_df = pd.DataFrame({"id": test_id, "money_room":test_pred})

In [53]:
submit_df.head(2)

,id,money_room
0,0,2.148840e+07
1,1,2.186247e+07


In [54]:
submit_file_path = "/tmp/sample_submit.csv"
submit_df.to_csv(submit_file_path, index=False, header=False)

In [ ]:
stage_name = "@model"

session.sql(f"""
    PUT file://{submit_file_path} {stage_name}
    OVERWRITE = TRUE
""").collect()